In [556]:
import warnings
warnings.simplefilter(action='ignore')

import datetime as dt
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [557]:
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', None)

In [558]:
from alpaca.data.requests import NewsRequest
from alpaca.data import StockHistoricalDataClient, TimeFrame 
from alpaca.data.requests import StockQuotesRequest, StockBarsRequest

from alpaca.trading.client import TradingClient

KEY = "PKQ9XJDJLTX686HE3ZL9"
SECRET = "4ES985YvYlOWz0eAtzdhcETJ1asEPBSw3gq9ZXs7"
ENDP = "https://paper-api.alpaca.markets"

In [559]:
import tensorflow as tf
import keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GlobalAveragePooling1D, Conv2D, ConvLSTM2D, ConvLSTM1D, Input, Flatten, Reshape, TextVectorization, concatenate


In [560]:
INTERVAL = 60
LOOKBACK = 50
STOCKS = 4

In [561]:
def build_model(time_steps, items):
    input_volatility = Input(shape=(time_steps, items), name="input_vol")
    input_civ = Input(shape=(1,), name="input_prem")
    input_piv = Input(shape=(1,), name="input piv")
    input_dte = Input(shape=(1,), name="input_dte")
    input_pc = Input(shape=(1,), name="input_pc")

    lstm1 = LSTM(units=32, return_sequences=True,input_shape=(time_steps, items))(input_volatility)
    lstm2 = LSTM(units=16, return_sequences=False)(lstm1)
    dense_lstm = Dense(units=10, activation="relu")(lstm2)
    concatted1 = concatenate([input_civ, input_piv,input_dte, input_pc])
    dense1 = Dense(units=1, activation="relu")(concatted1)
    concatted2 = concatenate([dense1, dense_lstm])
    dense2 = Dense(units=1, activation="sigmoid")(concatted2)
    model = keras.Model(inputs = [input_volatility, input_civ, input_piv,input_dte, input_pc], outputs=dense2)

    model.compile(optimizer="adam", loss="binary_crossentropy",  metrics=['accuracy'])

    return model

    

In [562]:
def collect_dfs(dir_path):
    directory_path = dir_path
    dfs = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            print(filename)
            file_path = os.path.join(directory_path, filename)

            # Read the text file into a DataFrame (adjust read_csv parameters based on your file format)
            df = pd.read_csv(file_path, sep=", ")
            df = df[(df["[STRIKE_DISTANCE]"] < 1) & (df["[DTE]"] > 25) & (df["[DTE]"] < 35)]
            # Append the DataFrame to the list
            dfs.append(df)
    full_df = pd.concat(dfs, ignore_index=True)
    return full_df

In [563]:
def get_data(stocks, start_date, end_date):
    data_client = StockHistoricalDataClient(KEY, SECRET)

    request_params = StockBarsRequest(
        symbol_or_symbols=stocks,
        timeframe=TimeFrame.Day,
        adjustment="split",
        start=start_date,
        end=end_date
        )



    bars_df = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)
    stock_data = pd.DataFrame(bars_df)
    # print(stock_data)
    stock_data = stock_data.groupby(['symbol', 'timestamp']).mean().unstack(level=0)
    open_prices = stock_data["open"]
    log_returns = np.log((open_prices.pct_change()+1).dropna())
    return log_returns, open_prices

In [564]:
def compute_vol(raw_data, interval):
   
    data = raw_data
    while(len(data)%interval != 0):
        data.drop(data.tail(1).index,inplace=True)
   
    # var_data = np.array(data[len(data)%interval:]).reshape(len(data)//interval, interval,-1).var(axis=1)
    var_data = data.rolling(interval).var()
    var_data = var_data.dropna()
    return np.sqrt(var_data)

Plan: 

For every option:
    - Get the vol of past 5 months before option was released
    - Get price now, get price at exp. Get profit (if profit positive, "long" else "short")

In [565]:
def build_train_data():
    options_data = collect_dfs("./aapl")
    # quit()


    today = dt.datetime.now()
    returns, prices = get_data(["AAPL", "MSFT", "GOOG"], today - dt.timedelta(7000), today)
    vol_inputs = []
    prem_inputs = []
    piv_inputs = []
    dte_inputs = []
    labels = []
    put_call_vol_ratios = []
    for index, row in options_data.iterrows():
        try:
            quote_date = row["[QUOTE_DATE]"]
            exp_date   = row["[EXPIRE_DATE]"]

            strike = row["[STRIKE]"]

            call_last = row["[C_LAST]"]
            put_last = row["[P_LAST]"]

            dte = row["[DTE]"]

            civ = row["[C_IV]"]
            piv = row["[P_IV]"]

            put_vol = row["[P_VOLUME]"]
            call_vol = row["[C_VOLUME]"]
          

            quote_price = prices.loc[quote_date, "AAPL"]
            exp_price = prices.loc[exp_date, "AAPL"]
            # print(type(strike))
        
          
            
            now = dt.datetime.strptime(quote_date, '%Y-%m-%d').date()
            start = now -dt.timedelta(3000)

            # print(prices.loc[start:now], 30)

            temp = compute_vol(prices.loc[start:now], INTERVAL)
            temp = temp[-LOOKBACK:]
            # print(temp)
            temp["avg"] = temp.mean(axis=1)
            # print("test", temp)
            temp = temp[["AAPL", "MSFT", "GOOG","avg"]]
            ratio = call_vol/put_vol
            put_call_vol_ratios.append(ratio)
            # print(temp)
            vol_inputs.append(temp)
            # prem_inputs.append(((call_last+put_last)/strike))
            prem_inputs.append(civ)
            piv_inputs.append(piv)
            dte_inputs.append(dte)

            if (exp_price-strike-call_last-put_last).item() > 0:
                labels.append(1)
            elif (strike-exp_price-call_last-put_last).item() > 0:
                labels.append(1)
            else:
                labels.append(0)
            
        except:
            continue
        # break
    # print(inputs)
    print(vol_inputs)
    return vol_inputs, prem_inputs, piv_inputs,dte_inputs, put_call_vol_ratios, labels




    

In [566]:
def train_model(model):
    inputs, prem_inputs,piv_inputs, dte_inputs, pc_ratio, labels = build_train_data()
    
    # labels = np.array(labels)
    # print(np.array(inputs).shape, np.array(labels).shape)
    # print(np.array(prem_inputs).shape, np.array(dte_inputs).shape)
    # print(inputs)
    
    inputs_train, inputs_test, prem_inputs_train, prem_inputs_test, piv_train, piv_test, dte_inputs_train, dte_inputs_test, pc_train, pc_test,y_train, y_test \
    = train_test_split(inputs, prem_inputs, piv_inputs,dte_inputs, pc_ratio, labels, test_size=0.2, random_state=42)
    

  
    model.fit(
        [np.array(inputs_train),
        np.array(prem_inputs_train),
        np.array(piv_train),
        np.array(dte_inputs_train),
        np.array(pc_train)],
        np.array(y_train),
        epochs=20,
        batch_size=32,
    )

    return inputs_test, prem_inputs_test, piv_test,dte_inputs_test, pc_test, y_test



In [567]:
def make_prediction(model, vol, prem, dte):
    return model.predict([vol, prem, dte])

In [568]:
def backtest(model):
   
    options_data = collect_dfs("./test")

    today = dt.datetime.now()
    returns, prices = get_data(["AAPL", "MSFT", "GOOG"], today - dt.timedelta(7000), today)
    score = 0
    count = 0
    profits = []
    predicts = []
    values = []
    for index, row in options_data.iterrows():
        try:
            quote_date = row["[QUOTE_DATE]"]
            exp_date   = row["[EXPIRE_DATE]"]

            strike = row["[STRIKE]"]

            call_last = row["[C_LAST]"]
            put_last = row["[P_LAST]"]

            dte = row["[DTE]"]

            civ = row["[C_IV]"]
            piv = row["[P_IV]"]

            put_vol = row["[P_VOLUME]"]
            call_vol = row["[C_VOLUME]"]


            quote_price = prices.loc[quote_date, "AAPL"]
            exp_price = prices.loc[exp_date, "AAPL"]
            # print(type(strike))
        
            
            now = dt.datetime.strptime(quote_date, '%Y-%m-%d').date()
            start = now -dt.timedelta(3000)

            # print(prices.loc[start:now], 30)
            temp = compute_vol(prices.loc[start:now], INTERVAL)
            temp = temp[-LOOKBACK:]
            temp["avg"] = temp.mean(axis=1)
            temp = temp[["AAPL", "MSFT", "GOOG","avg"]]
            # prem_input = (call_last+put_last)/strike
            prem_input = civ

            prediction = model.predict([np.array([temp]), np.array([prem_input]), np.array([piv]),np.array([dte]), np.array([call_vol/put_vol])])
            
            prediction = prediction.flatten()[0]
            values.append(prediction)
            print(prediction)
            if prediction > 0.5:
                prediction = 1
            else:
                prediction = 0
            
            
            if prediction == 1:
                profit = max((exp_price-strike-call_last-put_last).item(), (strike-exp_price-call_last-put_last).item())
            else:
                # profit = min((strike+put_last+call_last-exp_price).item(), (exp_price-strike+put_last+call_last).item())
                profit = max((exp_price-strike-call_last-put_last).item(), (strike-exp_price-call_last-put_last).item())*-1
            profits.append(profit)
            
            if (exp_price-strike-call_last-put_last).item() > 0:
                actual = 1
            elif (strike-exp_price-call_last-put_last).item() > 0:
                actual = 1
            else:
                actual = 0
            predicts.append((actual, prediction))
            count += 1
            if actual == prediction:
                score +=1
            
        except:
            continue    
        # break
    
    print("score:", score/count)
    print("count", count)
    print("profit", np.array(profits).mean())
    # print("cost", call_last+put_last)
    print(predicts)
    print(values)



In [569]:
def simple_test(model, X_test, y_test):
    print("this is a test")
    print(model.evaluate(X_test, y_test, batch_size=32, verbose=0))
    # print(results)

In [570]:
model = build_model(LOOKBACK,STOCKS)
inputs_test, prem_inputs_test, piv_test,dte_inputs_test, pc_test ,y_test = train_model(model)


# simple_test(model, [np.array(inputs_test), np.array(prem_inputs_test), np.array(dte_inputs_test)], np.array(y_test))
backtest(model)  

# print(input)



aapl_eod_202207.txt
aapl_eod_202301.txt
aapl_eod_202209.txt
aapl_eod_202212.txt
aapl_eod_202204.txt
aapl_eod_202303.txt
aapl_eod_202210.txt
aapl_eod_202202.txt
aapl_eod_202201.txt
aapl_eod_202208.txt
aapl_eod_202302.txt
aapl_eod_202205.txt
aapl_eod_202211.txt
aapl_eod_202203.txt
aapl_eod_202206.txt
[symbol                          AAPL       MSFT       GOOG        avg
timestamp                                                            
2022-03-29 00:00:00-04:00   6.923941  12.528770   5.387280   8.279997
2022-03-30 00:00:00-04:00   6.942415  11.814730   5.348510   8.035219
2022-03-31 00:00:00-04:00   6.793393  11.073410   5.269428   7.712077
2022-04-01 00:00:00-04:00   6.663419  10.627261   5.186334   7.492338
2022-04-04 00:00:00-04:00   6.692336  10.570287   5.221136   7.494586
2022-04-05 00:00:00-04:00   6.780472  10.550845   5.303183   7.544833
2022-04-06 00:00:00-04:00   6.805497  10.499410   5.316601   7.540503
2022-04-07 00:00:00-04:00   6.793757  10.355531   5.310209   7.486499